<a href="https://colab.research.google.com/github/gauravshetty98/Gaurav-GIS-Repo/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Extraction and Cleaning

This notebook represents the steps to download the zip file from GitHub repository and extract the data from its content and create a single dataset.

Dataset :  https://www.samhsa.gov/data/report/2021-nsduh-state-specific-tables

The dataset contains state-wise PDFs with drug abuse and mental health information of the respective state.

We start of by installing `tabula` library. This library is used to extract the tables from the PDFs.


In [1]:
!pip install -q tabula-py

Once the library is installed, we move on to import all the required libraries. We make use of google drive to store the PDFs present in the dataset.

**This notebook requires your google drive mounting to store dataset files. You can delete those files after running the notebook**



In [2]:
import os
from os import path
import tabula
import pandas as pd
import numpy as np

from google.colab import drive

drive.mount("/content//gdrive")

Drive already mounted at /content//gdrive; to attempt to forcibly remount, call drive.mount("/content//gdrive", force_remount=True).


### Creating folders for storing all the data

`2021NSDUHsaeSpecificStatesTabs122022` is the name of the new folder. This folder will contain all the state wise PDFs.
A `tables` folder is also present, which will contain all the tables extracted from each state PDF.



In [3]:
dir = "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"
folder_name = "tables"

Creating a new folder if it is not present

In [4]:
if os.path.exists(dir) == False:
    os.mkdir(dir)
    print("done")
if os.path.exists(dir+"//"+folder_name) == False:
    os.mkdir(dir+"//"+folder_name)
    print("done")

!ls "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"

done
done
tables


Downloading the zip file from GitHub and extracting it into the drive folder we created

In [5]:
!wget "https://github.com/gauravshetty98/Gaurav-GIS-Repo/raw/main/2021NSDUHsaeSpecificStatesTabs122022.zip"
!unzip /content/2021NSDUHsaeSpecificStatesTabs122022.zip -d "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"

--2023-09-14 16:40:40--  https://github.com/gauravshetty98/Gaurav-GIS-Repo/raw/main/2021NSDUHsaeSpecificStatesTabs122022.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gauravshetty98/Gaurav-GIS-Repo/main/2021NSDUHsaeSpecificStatesTabs122022.zip [following]
--2023-09-14 16:40:40--  https://raw.githubusercontent.com/gauravshetty98/Gaurav-GIS-Repo/main/2021NSDUHsaeSpecificStatesTabs122022.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13301032 (13M) [application/zip]
Saving to: ‘2021NSDUHsaeSpecificStatesTabs122022.zip.7’

2021NSDUHsaeSpecifi 100%[===================>]  12.68M  83.9MB/s  

### Concatenating all files and creating a single DataFrame

We iterate through all PDF files present in the new directory and extract the tables present in the first two pages using `tabula.read_pdf()`

We then concatenate the 2 extracted tables into a single dataframe and add a new column `states` to the dataframe.

Finally, we convert the dataframe into csv and store it in the `tables` folder in google drive

In [6]:
ext = ('.pdf','.PDF')
final_table = pd.DataFrame()
for files in os.listdir(dir):
    if files.endswith(ext):
        print(files)
        tables = tabula.read_pdf(dir + "//" + files,pages=[1,2]) #address of pdf file
        filename = files.replace("NSDUHsae","").replace(".pdf","").replace("2021","")
        df1 = pd.DataFrame(tables[0])
        df2 = pd.DataFrame(tables[1])
        final_table = pd.concat([df1,df2])
        final_table['states'] = filename
        #print(final_table)
        final_table.to_csv(os.path.join(dir + "//" + folder_name, filename+".csv"), index=False)
        print(filename)
    else:
        print("Else: ", files)
#final_table.to_csv(os.path.join(folder_name, "FinalTable.csv"), index=False)

Else:  tables
Else:  images
NSDUHsaeAlabama2021.pdf
Alabama
NSDUHsaeAlaska2021.pdf
Alaska
NSDUHsaeArizona2021.pdf
Arizona
NSDUHsaeArkansas2021.pdf
Arkansas
NSDUHsaeCalifornia2021.pdf
California
NSDUHsaeColorado2021.pdf
Colorado
NSDUHsaeConnecticut2021.pdf
Connecticut
NSDUHsaeDelaware2021.pdf
Delaware
NSDUHsaeDistrictOfCol2021.pdf
DistrictOfCol
NSDUHsaeFlorida2021.pdf
Florida
NSDUHsaeGeorgia2021.pdf
Georgia
NSDUHsaeHawaii2021.pdf
Hawaii
NSDUHsaeIdaho2021.pdf
Idaho
NSDUHsaeIllinois2021.pdf
Illinois
NSDUHsaeIndiana2021.pdf
Indiana
NSDUHsaeIowa2021.pdf
Iowa
NSDUHsaeKansas2021.pdf
Kansas
NSDUHsaeKentucky2021.pdf
Kentucky
NSDUHsaeLouisiana2021.pdf
Louisiana
NSDUHsaeMaine2021.pdf
Maine
NSDUHsaeMaryland2021.pdf
Maryland
NSDUHsaeMassachusetts2021.pdf
Massachusetts
NSDUHsaeMichigan2021.pdf
Michigan
NSDUHsaeMidwest2021.pdf
Midwest
NSDUHsaeMinnesota2021.pdf
Minnesota
NSDUHsaeMississippi2021.pdf
Mississippi
NSDUHsaeMissouri2021.pdf
Missouri
NSDUHsaeMontana2021.pdf
Montana
NSDUHsaeNational2021.pdf
N

Output shows all the CSVs extracted and stored state wise in the tables folder

In [7]:
!ls "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022/tables"

Alabama.csv	   Indiana.csv	      National.csv	 RhodeIsland.csv
Alaska.csv	   Iowa.csv	      Nebraska.csv	 SouthCarolina.csv
Arizona.csv	   Kansas.csv	      Nevada.csv	 South.csv
Arkansas.csv	   Kentucky.csv       NewHampshire.csv	 SouthDakota.csv
California.csv	   Louisiana.csv      NewJersey.csv	 Tennessee.csv
Colorado.csv	   Maine.csv	      NewMexico.csv	 Texas.csv
Connecticut.csv    Maryland.csv       NewYork.csv	 Utah.csv
Delaware.csv	   Massachusetts.csv  NorthCarolina.csv  Vermont.csv
DistrictOfCol.csv  Michigan.csv       NorthDakota.csv	 Virginia.csv
Florida.csv	   Midwest.csv	      Northeast.csv	 Washington.csv
Georgia.csv	   Minnesota.csv      Ohio.csv		 West.csv
Hawaii.csv	   Mississippi.csv    Oklahoma.csv	 WestVirginia.csv
Idaho.csv	   Missouri.csv       Oregon.csv	 Wisconsin.csv
Illinois.csv	   Montana.csv	      Pennsylvania.csv	 Wyoming.csv


All the CSVs are concatenated into a single dataframe.

In [8]:
final_dataset = pd.DataFrame()
csv_path = dir + "//" + folder_name
for tables in os.listdir(csv_path):
  df3 = pd.read_csv(csv_path + "//" + tables)
  final_dataset = pd.concat([final_dataset,df3])

In [9]:
print(final_dataset.head())
print(final_dataset.shape)

                                             Measure    12+ 12-17 18-25  26+  \
0                                      ILLICIT DRUGS    NaN   NaN   NaN  NaN   
1              Illicit Drug Use in the Past Month1,2    403    17    98  288   
2                     Marijuana Use in the Past Year    537    30   152  355   
3                    Marijuana Use in the Past Month    315    16    90  209   
4  Perceptions of Great Risk from Smoking Marijua...  1,128   105    83  940   

     18+   states  
0    NaN  Alabama  
1    386  Alabama  
2    507  Alabama  
3    299  Alabama  
4  1,023  Alabama  
(2408, 7)


---------------------
## Data Cleaning

Here we start with the data cleaning. We first replace all the `NaN` values with 0.

In [10]:
final_dataset = final_dataset.fillna(0)
print(final_dataset.head())

                                             Measure    12+ 12-17 18-25  26+  \
0                                      ILLICIT DRUGS      0     0     0    0   
1              Illicit Drug Use in the Past Month1,2    403    17    98  288   
2                     Marijuana Use in the Past Year    537    30   152  355   
3                    Marijuana Use in the Past Month    315    16    90  209   
4  Perceptions of Great Risk from Smoking Marijua...  1,128   105    83  940   

     18+   states  
0      0  Alabama  
1    386  Alabama  
2    507  Alabama  
3    299  Alabama  
4  1,023  Alabama  


### Dropping Empty Rows

The dataset contains some rows which are empty and dont contain any necessary information. Here we search for those rows and drop them from the dataset

In [11]:
row_list = list([])
count = 0
for i in range(0, final_dataset.shape[0]):
  if list(final_dataset.iloc[i,1:6]) == list([0,0,0,0,0]):
    final_dataset.iloc[i+1,0] = str(final_dataset.iloc[i,0]) + " " + str(final_dataset.iloc[i+1,0])
    print("i: ",i, "Shape: ", final_dataset.shape)
    row_list.append(i)
    count = count + 1
    #print(final_dataset.iloc[i,:])
    #final_dataset.drop([i], inplace= True)

print(row_list)
print(count)
# new_df = pd.DataFrame([row for i, row in final_dataset.iterrows() if final_dataset.iloc[i,1:6] != list([0,0,0,0,0])])
# new_df.head()


i:  0 Shape:  (2408, 7)
i:  5 Shape:  (2408, 7)
i:  15 Shape:  (2408, 7)
i:  18 Shape:  (2408, 7)
i:  22 Shape:  (2408, 7)
i:  26 Shape:  (2408, 7)
i:  34 Shape:  (2408, 7)
i:  43 Shape:  (2408, 7)
i:  48 Shape:  (2408, 7)
i:  58 Shape:  (2408, 7)
i:  61 Shape:  (2408, 7)
i:  65 Shape:  (2408, 7)
i:  69 Shape:  (2408, 7)
i:  77 Shape:  (2408, 7)
i:  86 Shape:  (2408, 7)
i:  91 Shape:  (2408, 7)
i:  101 Shape:  (2408, 7)
i:  104 Shape:  (2408, 7)
i:  108 Shape:  (2408, 7)
i:  112 Shape:  (2408, 7)
i:  120 Shape:  (2408, 7)
i:  129 Shape:  (2408, 7)
i:  134 Shape:  (2408, 7)
i:  144 Shape:  (2408, 7)
i:  147 Shape:  (2408, 7)
i:  151 Shape:  (2408, 7)
i:  155 Shape:  (2408, 7)
i:  163 Shape:  (2408, 7)
i:  172 Shape:  (2408, 7)
i:  177 Shape:  (2408, 7)
i:  187 Shape:  (2408, 7)
i:  190 Shape:  (2408, 7)
i:  194 Shape:  (2408, 7)
i:  198 Shape:  (2408, 7)
i:  206 Shape:  (2408, 7)
i:  215 Shape:  (2408, 7)
i:  220 Shape:  (2408, 7)
i:  230 Shape:  (2408, 7)
i:  233 Shape:  (2408, 7)
i:  

In [17]:
for i in range(0,final_dataset.sha)
df_new.loc[len(df_new)] = final_dataset.iloc[i,:]

final_dataset = final_dataset.drop(row_list, axis = 0, inplace = True)
print(final_dataset.shape)
#print(type(row_list))

KeyError: ignored

In [ ]:
print(final_dataset.iloc[15,:])

In [ ]:
final_dataset.head()

### Replacing more Null values

The dataset still contains some null values. During the extraction process the null values in the PDF were converted to string `"--"`. We search for this string and replace it with zero.

In [ ]:
print("Before: ", final_dataset.iloc[12,:])
final_dataset = final_dataset.replace("--",0)
print("After: ", final_dataset.iloc[12,:])

--------
### Storing the dataset into a CSV file

In [ ]:
final_dataset.to_csv(os.path.join(dir + "//" + folder_name, "state_dataset.csv"), index=False)